In [ ]:
# 라이브러리 import

import jchm
import cv2
import os
import time
import math
import shutil

from http.server import ThreadingHTTPServer, SimpleHTTPRequestHandler
from functools import partial

In [ ]:
# output 디렉토리 초기화 - 없으면 생성, 있으면 내부 jpg 파일 삭제

if os.path.exists("./output"):
    listdir = os.listdir("./output")
    for f in listdir:
        if f.endswith(".jpg"):
            os.remove(f"./output/{f}")
else:
    os.makedirs("./output")

In [ ]:
# 카메라 입력 받아온 후 녹화

OUTPUT_DIR = "./output"
FPS = 30
DT = 1.0 / FPS

next_frame = time.perf_counter()

cnt = 0
while True:
    next_frame += DT
    now = time.perf_counter()
    if now < next_frame:
        time.sleep(next_frame - now)
    else:
        missed = math.floor((now - next_frame) / DT)
        next_frame += missed * DT
    cnt += 1
    img = jchm.camera.get_image()
    jchm.camera.show_image(img, "center")
    cv2.imwrite(f"{OUTPUT_DIR}/{cnt}.jpg", img, [cv2.IMWRITE_JPEG_QUALITY, 80])

In [ ]:
# output 디렉토리 압축

!cd output && find . -exec touch -t 200001010000 {} \;

shutil.make_archive("./output", "zip", "./output/")

In [ ]:
# 파일 HTTP 정적 서빙
# 동글 연결된 디바이스에서 127.17.0.3로 연결

PORT = 8000
BIND = "0.0.0.0"
DIR = os.getcwd()

def main():
  handler_cls = partial(SimpleHTTPRequestHandler, directory=DIR)
  with ThreadingHTTPServer((BIND, PORT), handler_cls) as httpd:
    root = os.path.abspath(DIR)
    print(f"Serving {root} at http://{BIND}:{PORT}")
    try:
      httpd.serve_forever()
    except KeyboardInterrupt:
      pass

if __name__ == "__main__":
  main()